In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import sent2vec

In [2]:
user_df = pd.read_csv('data/health_experts50.csv')
##id, tweet, user
user_df

,tweetid,screen_name,date,lang,text,state,friends_count,followers_count
0,1301875815882412032,HelenBranswell,Fri Sep 04 13:32:36 +0000 2020,en,Interesting #Covid19 vaccine development: Scie...,MA,2079,225280
1,1326359181813755904,MoNscience,Wed Nov 11 03:00:46 +0000 2020,en,Followers of this space might recall that I po...,NaN,4467,13549
2,1337244117353508864,JoeBrunoWSOC9,Fri Dec 11 03:53:37 +0000 2020,en,Bishop who battled COVID-19 says he thought Go...,NC,4017,48909
3,1382829830777958400,EricTopol,Thu Apr 15 22:54:58 +0000 2021,en,Risk of covid reinfection in a prospective stu...,CA,556,686676
4,1395752514222530560,ghn_news,Fri May 21 14:45:05 +0000 2021,en,Tunisia has ended its 1-week lockdown; despite...,MD USA,2268,16655
...,...,...,...,...,...,...,...,...
7093,1329494791344812032,DemFromCT,Thu Nov 19 18:40:33 +0000 2020,zxx,https://t.co/9GOY17jaQb,CT,2318,23508
7094,1299143481244487680,luckytran,Fri Aug 28 00:35:17 +0000 2020,en,A mass rally at the White House with no social...,New York City and Australia,3937,56156
7095,1232308398944854016,NACCHOalerts,Tue Feb 25 14:16:32 +0000 2020,zxx,https://t.co/2SOmlwwBHf,DC,3522,28576
7096,1292615809383071744,rwang0,Mon Aug 10 00:16:39 +0000 2020,en,MyPOV: if protestors can get out there churchg...,Silicon Valley | SF | FL 410+,103619,141982


In [3]:
##Clean up text to remove special characters
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
from string import digits
import re
irrelevant_chars="~?!./\:;+=&^%$#@(,)[]_*"
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def deep_clean(x):
    x=x.lower()
    x=re.sub(r'http\S+', '', x)
    remove_digits = str.maketrans(' ', ' ', digits)
    remove_chars = str.maketrans(' ', ' ', irrelevant_chars)
    x = x.translate(remove_digits)
    x = x.translate(remove_chars)
    x = emoji_pattern.sub(r'', x)
    x=x.replace("-",' ')
    x=x.replace('!',' ')
    x=x.replace('?',' ')
    x=x.replace('@',' ')
    x=x.replace('&',' ')
    x=x.replace('$',' ')
    x=x.replace('``',' ')
    x=x.replace("'s",' ')
    x=x.replace("''",' ')
    return x

In [4]:
##NOTE: Students will need to group individual tweets by the user. Here's a snippet:
user_text={}
for i in range(len(user_df)): #Here df is the tweet data.
    user=user_df['screen_name'].iloc[i]
    text=user_df['text'].iloc[i]
    if user not in user_text:
        user_text[user]=' '
    user_text[user]=deep_clean(str(user_text[user])+' '+str(text))

user_text_df=pd.DataFrame(user_text.items(),columns=['screen_name','cleaned_text'])

In [5]:
#https://github.com/epfml/sent2vec -> download sent2vec_twitter_bigrams
model = sent2vec.Sent2vecModel()
model.load_model('twitter_bigrams.bin')

embeddings = model.embed_sentences(user_text_df['cleaned_text'].tolist())
e=embeddings.tolist()

user_text_df['embeddings']=e
print(len(user_text_df))

48


In [6]:
filename = 'model.sav'
model=pickle.load(open(filename, 'rb'))

/Users/paiheng/miniconda3/envs/General/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
y_pred=model.predict(np.asarray(user_text_df['embeddings'].tolist()))
y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
y_pred_prob = model.predict_proba(np.asarray(user_text_df['embeddings'].tolist()))
y_pred_prob

array([[0.15909044, 0.84090956],
       [0.21489008, 0.78510992],
       [0.14196902, 0.85803098],
       [0.12492592, 0.87507408],
       [0.08997536, 0.91002464],
       [0.12488939, 0.87511061],
       [0.152878  , 0.847122  ],
       [0.11231565, 0.88768435],
       [0.02918177, 0.97081823],
       [0.26998049, 0.73001951],
       [0.12482156, 0.87517844],
       [0.32095513, 0.67904487],
       [0.11176967, 0.88823033],
       [0.08815855, 0.91184145],
       [0.08539698, 0.91460302],
       [0.33385485, 0.66614515],
       [0.18605773, 0.81394227],
       [0.05123814, 0.94876186],
       [0.06016279, 0.93983721],
       [0.06420794, 0.93579206],
       [0.10490102, 0.89509898],
       [0.10505903, 0.89494097],
       [0.12205887, 0.87794113],
       [0.0625888 , 0.9374112 ],
       [0.07707782, 0.92292218],
       [0.20815296, 0.79184704],
       [0.11762274, 0.88237726],
       [0.04234982, 0.95765018],
       [0.20832524, 0.79167476],
       [0.16342719, 0.83657281],
       [0.